## Odwzorowanie Gaussa-Krügera: układy współrzędnych płaskich stosowanych w Polsce

Autor: **Radosław Dąbkowski**

Numer indeksu: **325683**

Grupa: **1**
Numer: **2**

### Wstęp

Celem tego zadania było zapoznanie się z układami współrzędnych płaskich stosowanych w Polsce oraz porównanie ich do innych układów.

Zadanie 1. Przeliczenie współrzędnych z ćwiczenia 3. do układów PL-1992
i PL-2000

In [46]:
import numpy as np
from pyproj import Transformer, CRS, Geod

# phi = 52.005893
# lam = 18.571495
# input_proj = CRS.from_epsg(2180)
# output_proj = CRS.from_epsg(4326)


# x, y = Transformer.from_proj(input_proj, output_proj).transform(phi, lam)
# print(x, y)

nr = 2
m0 = 0.999923
phi1 = 50 + nr*15/60
lam1 = 18 + nr*15/60

geod = Geod(ellps='GRS80')

d12 = 40e3
a12 = 0
d23 = 100e3
a23 = 90
d34 = 40e3
a34 = 180

p1 = (lam1,phi1,0)
p2 = geod.fwd(lam1,phi1,a12,d12)
p3 = geod.fwd(p2[0],p2[1],a23,d23)
p4 = geod.fwd(p3[0],p3[1],a34,d34)


# Transformacja współrzędnych geodezyjnych na płaszczyźnie Gaussa-Krügera

input_proj = CRS.from_epsg(4326)
output_proj = CRS.from_epsg(2180)
philam_2000_transformer = Transformer.from_proj(input_proj, output_proj)


pl2000_1 = philam_2000_transformer.transform(p1[1], p1[0])
pl2000_2 = philam_2000_transformer.transform(p2[1], p2[0])
pl2000_3 = philam_2000_transformer.transform(p3[1], p3[0])
pl2000_4 = philam_2000_transformer.transform(p4[1], p4[0])

pl2000_points = [pl2000_1, pl2000_2, pl2000_3, pl2000_4]


pl1992_1 = (pl2000_1[0] - 5300000, pl2000_1[1] - nr*1000000)
pl1992_2 = (pl2000_2[0] - 5300000, pl2000_2[1] - nr*1000000)
pl1992_3 = (pl2000_3[0] - 5300000, pl2000_3[1] - nr*1000000)
pl1992_4 = (pl2000_4[0] - 5300000, pl2000_4[1] - nr*1000000)

pl1992_points = [pl1992_1, pl1992_2, pl1992_3, pl1992_4]

print('Numer punktu\tx1992\t\t\ty1992\t\t\tx2000\t\ty2000')
for i in range(len(pl1992_points)):
    print(f'{i+1}\t\t{pl1992_points[i][0]:.3f}\t\t{pl1992_points[i][1]:.3f}\t\t{pl2000_points[i][0]:.3f}\t{pl2000_points[i][1]:.3f}')


Numer punktu	x1992			y1992			x2000		y2000
1		-5007334.189		-1535449.803		292665.811	464550.197
2		-4967362.487		-1535179.945		332637.513	464820.055
3		-4968038.862		-1435250.943		331961.138	564749.057
4		-5008009.854		-1434754.394		291990.146	565245.606


Zadanie 2. Redukcje odwzorowawcze

In [47]:
# 1. Oblicz długości odcinków między punktami, w odpowiednich parach: 1-2, 2-3, 3-4, 4-1, na
# płaszczyźnie układu PL-2000 (z tw. Pitagorasa, sukład);

d12 = np.sqrt((pl2000_2[0] - pl2000_1[0])**2 + (pl2000_2[1] - pl2000_1[1])**2)
d23 = np.sqrt((pl2000_3[0] - pl2000_2[0])**2 + (pl2000_3[1] - pl2000_2[1])**2)
d34 = np.sqrt((pl2000_4[0] - pl2000_3[0])**2 + (pl2000_4[1] - pl2000_3[1])**2)
d41 = np.sqrt((pl2000_1[0] - pl2000_4[0])**2 + (pl2000_1[1] - pl2000_4[1])**2)

# 2. Wykonaj redukcje długości z płaszczyzny układu PL-2000 na elipsoidę:
# 1. Oblicz długość odcinka na płaszczyźnie G-K (sgk):
# • sgk = sukład/mukład
# 0 ,
# • albo z tw. Pitagorasa na podstawie współrzędnych płaskich punktów na płaszczyźnie
# G-K.

sgk12 = d12/m0
sgk23 = d23/m0
sgk34 = d34/m0
sgk41 = d41/m0

# 2. Oblicz redukcję długości:
# rAB = sAB · y2
# A + yAyB + y2
# B
# 6R2
# m
# (5)
# gdzie: Rm – średni promień krzywizny dla odcinka (Rm = √Nm · Mm obliczone dla φm
# – szerokość geodezyjna środkowego punktu odcinka); y – współrzędne y obu punktów na
# płaszczyźnie G-K; sAB – długość odcinka (dowolnie, czy na płaszczyźnie, czy na elipso-
# idzie)


# Współrzędne geodezyjne środkowego punktu odcinka
phi_m12 = (p1[1] + p2[1])/2
phi_m23 = (p2[1] + p3[1])/2
phi_m34 = (p3[1] + p4[1])/2
phi_m41 = (p4[1] + p1[1])/2

# Główne promienie krzywizny

# Promień przekroju południkowego (podłużnego)
# M = a ( 1 − e 2 ) ( 1 − e 2 sin 2 ⁡ B ) 3 {\displaystyle M={\frac {a(1-e^{2})}{\sqrt {(1-e^{2}\sin ^{2}B)^{3}}}}}
# Promień przekroju pierwszego wertykału (poprzecznego)
# N = a 1 − e 2 sin 2 ⁡ B {\displaystyle N={\frac {a}{\sqrt {1-e^{2}\sin ^{2}B}}}}
# https://pl.wikipedia.org/wiki/Elipsoida_ziemska#Główne_promienie_krzywizny

def calculate_n_m(phi, a=6378137, e=0.0818191910428158) -> tuple:
    B = np.radians(phi)
    M = a * (1 - e ** 2) / np.sqrt((1 - e ** 2 * np.sin(B) ** 2) ** 3)
    N = a / np.sqrt(1 - e ** 2 * np.sin(B) ** 2)
    n = M / N
    m = N / np.sqrt(1 - e ** 2 * np.sin(B) ** 2)
    return n, m

nm12, mm12 = calculate_n_m(phi_m12)
nm23, mm23 = calculate_n_m(phi_m23)
nm34, mm34 = calculate_n_m(phi_m34)
nm41, mm41 = calculate_n_m(phi_m41)

# Redukcja długości odcinków

r12 = d12 * pl2000_1[1] * pl2000_2[1] / (6 * mm12**2)
r23 = d23 * pl2000_2[1] * pl2000_3[1] / (6 * mm23**2)
r34 = d34 * pl2000_3[1] * pl2000_4[1] / (6 * mm34**2)
r41 = d41 * pl2000_4[1] * pl2000_1[1] / (6 * mm41**2)

# 3. Oblicz długość odcinka na elipsoidzie:
# • sAB = sgk − rAB

s12 = sgk12 - r12
s23 = sgk23 - r23
s34 = sgk34 - r34
s41 = sgk41 - r41

from pyproj import Geod
geod = Geod(ellps='GRS80')

s12_inv = geod.inv(p1[0], p1[1], p2[0], p2[1])[2]
s23_inv = geod.inv(p2[0], p2[1], p3[0], p3[1])[2]
s34_inv = geod.inv(p3[0], p3[1], p4[0], p4[1])[2]
s41_inv = geod.inv(p4[0], p4[1], p1[0], p1[1])[2]

# print out results
print('Numer odcinka\t\tDługość odcinka z zadania 3 [m]\tZredukowana długość odcinka [m]')
print(f'1-2\t\t\t{d12:.3f}\t\t\t{s12:.3f}')
print(f'2-3\t\t\t{d23:.3f}\t\t\t{s23:.3f}')
print(f'3-4\t\t\t{d34:.3f}\t\t\t{s34:.3f}')
print(f'4-1\t\t\t{d41:.3f}\t\t\t{s41:.3f}')

Numer odcinka		Długość odcinka z zadania 3 [m]	Zredukowana długość odcinka [m]
1-2			39972.613			39940.611
2-3			99931.291			99832.376
3-4			39974.076			39925.292
4-1			100697.676			100597.961


Zredukowane długości są niewiele mniejsze od długości policzonych za pomocą zadania odwrotnego. Różnice te wzrastają wraz z długościami odcinków.

Zadanie 3. Obliczenie pola powierzchni "trapezu" na płaszczyźnie

In [48]:
def pole_wielokata(points: list) -> float:
    p = 0
    for i in range(1, len(points) - 1):
        p += points[i][0] * (points[i+1][1] - points[i-1][1])
    # last element
    p += points[-1][0] * (points[0][1] - points[-2][1])
    # first element
    p += points[0][0] * (points[1][1] - points[-1][1])
    return abs(p/2)


pole_trapezu_1992 = pole_wielokata(pl1992_points)
pole_trapezu_2000 = pole_wielokata(pl2000_points)

pl_laea = CRS.from_epsg(3035)
philam_laea_transformer = Transformer.from_proj(input_proj, pl_laea)

pl_laea_1 = philam_laea_transformer.transform(p1[1], p1[0])
pl_laea_2 = philam_laea_transformer.transform(p2[1], p2[0])
pl_laea_3 = philam_laea_transformer.transform(p3[1], p3[0])
pl_laea_4 = philam_laea_transformer.transform(p4[1], p4[0])

pl_laea_points = [pl_laea_1, pl_laea_2, pl_laea_3, pl_laea_4]

pole_trapezu_laea = pole_wielokata(pl_laea_points)

print(f'Pole trapezu w układzie PL-1992: {pole_trapezu_1992:.2f} m2')
print(f'Pole trapezu w układzie PL-2000: {pole_trapezu_2000:.2f} m2')
print(f'Pole trapezu w układzie PL-LAEA: {pole_trapezu_laea:.2f} m2')

Pole trapezu w układzie PL-1992: 4009537314.64 m2
Pole trapezu w układzie PL-2000: 4009537314.64 m2
Pole trapezu w układzie PL-LAEA: 4014988079.56 m2


Pola trapezu obliczone w układzie PL-1992 oraz PL-2000 wyniosły tyle samo. Brak różnic wynika z tego, że oba układy są oparte na tym samym przesuniętym odwzorowaniu.

Pole trapezu w układzie PL-LAEA jest większe niż w pozostałych układach. Wynika to z tego, że układ PL-LAEA jest układem równopolowym, a pole obliczone w tym układzie jest najbliższe rzeczywistemu.